This notebook uses the Scientific Python (scipy) stack tools to generate flow duration curves from current USGS NWIS data.

Using recipes from this notebook, you can make:
* USGS Station Summaries
* Flow duration curves
* Iterative import and compilation of USGS station information and data
* boxplots using pandas
* iterative charts (one monthly summary boxplot per station)
* Gantt charts of USGS stations

## Background

Check out this for some great `pandas` applications:
http://earthpy.org/time_series_analysis_with_pandas_part_2.html

In [1]:
%matplotlib inline
import pandas as pd
import platform
import sys
import xmltodict
import numpy as np
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Pandas Version " + str(pd.__version__))
print("Numpy Version " + str(np.__version__))
print("Matplotlib Version " + str(matplotlib.__version__))

Operating System Linux 3.19.0-43-generic
Python Version 2.7.6 (default, Jun 22 2015, 17:58:13) 
[GCC 4.8.2]
Pandas Version 0.17.1
Numpy Version 1.10.4
Matplotlib Version 1.5.1


This following the suggested import call for the well application function.

In [3]:
import wellapplication as wa

In [4]:
wa.__version__

'0.1.5'

Call function class usgs and assign it as USGS.  This will allow for the implementation of all of the usgs functions.  This function class allows for the import of usgs data.

In [5]:
USGS = wa.usgs()

Import list of stations based on Cache Valley HUCS. The function `getStationsfromHUC` will import a list of stations from a list of HUCs.

In [6]:
stations = USGS.getStationsfromHUC(str('16010203,16010202'))

Import site data from HUCS. The function `getStationsInfoFromHUC` will import a list of stations information from a list of HUCs and save it in a Pandas DataFrame.

In [7]:
siteinfo = USGS.getStationInfoFromHUC(str('16010203,16010202'))

Inspect columns of imported site data

In [8]:
siteinfo.columns

Index([u'agency_cd', u'site_no', u'station_nm', u'site_tp_cd', u'dec_lat_va',
       u'dec_long_va', u'coord_acy_cd', u'dec_coord_datum_cd', u'alt_va',
       u'alt_acy_va', u'alt_datum_cd', u'huc_cd'],
      dtype='object')

Reproject Lat Long into UTM X and Y.  These functions use an import from the `pyproj` package.

In [9]:
siteinfo['UTM_X'] = siteinfo[['dec_long_va','dec_lat_va']].apply(lambda x: wa.avgMeths.projx(x),1)
siteinfo['UTM_Y']= siteinfo[['dec_long_va','dec_lat_va']].apply(lambda x: wa.avgMeths.projy(x),1)

Use the `getelev` function to call the point elevation service of the USGS. The Elevation Point Query Service <a href=http://ned.usgs.gov/epqs/> EPQS </a>

In [ ]:
siteinfo['Elev'] = siteinfo[['dec_long_va','dec_lat_va']].apply(lambda x: USGS.getelev(x),1)

The `getWLfromHUC` function downloads all of the data for a huc from the <a href=http://waterservices.usgs.gov/rest/GW-Levels-Test-Tool.html> USGS Groundwater Service Tool</a> and outputs it into a Pandas DataFrame.

In [10]:
HUClist = ['16010203','16010202']
HUClist = USGS.parsesitelist(HUClist)

In [11]:
data = USGS.getWLfromHUC(HUClist)

The `cleanGWL` function does a simple query on the USGS Water Level DataFrame to remove all of the <a href=http://help.waterdata.usgs.gov/codes-and-parameters/water-level-site-status-codes-lev_status_cd> lev_status_cd</a> value codes that signify error.

In [12]:
data = USGS.cleanGWL(data)

In [13]:
data

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd
0,USGS,413125111484701,GW,1968-10-24,NaN,NaN,2.00,NaN,NaN,NaN,NaN,D,2,S,E,A
1,USGS,413142111480801,GW,1968-10-23,NaN,NaN,16.00,NaN,NaN,NaN,NaN,D,2,S,S,A
2,USGS,413157111484501,GW,1937-01-17,NaN,NaN,29.50,NaN,NaN,NaN,NaN,D,2,U,U,A
3,USGS,413157111484501,GW,1939-10-03,NaN,NaN,31.23,NaN,NaN,NaN,NaN,D,2,U,U,A
4,USGS,413157111484501,GW,1940-02-07,NaN,NaN,30.66,NaN,NaN,NaN,NaN,D,2,U,U,A
5,USGS,413157111484501,GW,1940-05-01,NaN,NaN,31.13,NaN,NaN,NaN,NaN,D,2,U,U,A
6,USGS,413157111484501,GW,1944-04-12,NaN,NaN,29.85,NaN,NaN,NaN,NaN,D,2,U,U,A
7,USGS,413157111484501,GW,1948-04-01,NaN,NaN,28.50,NaN,NaN,NaN,NaN,D,2,U,U,A
8,USGS,413157111484501,GW,1938-04-19,NaN,NaN,28.25,NaN,NaN,NaN,NaN,D,2,U,U,A
9,USGS,413157111484501,GW,1952-10-22,NaN,NaN,25.91,NaN,NaN,NaN,NaN,D,2,U,U,A


In [ ]:
stationWL['wlelev'] = stationWL[['lev_va','Elev']].apply(lambda x: wa.avgMeths.getwlelev(x),1)

In [ ]:
stationWL['date'], stationWL['Year'], stationWL['Month'] = zip(*stationWL['lev_dt'].apply(lambda x: wa.avgMeths.getyrmnth(x),1))

In [ ]:
-


try using diff function to get average monthly changes over time

In [ ]:
hucPlot(16030006) #cedar city valley

In [ ]:
hucPlot(16020204) #salt lake valley

In [ ]:
stationWL.reset_index(inplace=True)
stationWL.set_index('date',inplace=True)

In [ ]:
wlavgs = stationWL[['Year','Month','site_no','lev_va','UTM_X','UTM_Y']].groupby(['Year','Month','site_no']).mean()

In [ ]:
grpstat = stationWL.groupby('site_no')['lev_va'].agg([np.std,np.mean,np.median, np.min,np.max,np.size]).reset_index()

In [ ]:
USGS_Site_Inf = stationWL.groupby('site_no')['lev_dt'].agg([np.min,np.max,np.size]).reset_index()

In [ ]:
siteinfo.reset_index(inplace=True)

In [ ]:
grpst = pd.merge(grpstat, USGS_Site_Inf, on='site_no', how='left')

In [ ]:
grpsta = pd.merge(grpst, siteinfo, on='site_no', how='left')
avgwls = grpsta.drop_duplicates()
#avgwls.to_csv('E:\\PROJECTS\\UMAR\\Data\\USGS\\AvgWLs_USGS.csv')

In [ ]:
USGS_Site_Info = USGS_Site_Inf[USGS_Site_Inf['size']>50]

In [ ]:
wlLong = stationWL[stationWL['site_no'].isin(list(USGS_Site_Info['site_no'].values))]

In [ ]:
wlLongStats = pd.merge(wlLong,grpstat, on='site_no', how='left')

In [ ]:
wlLongStats['levDiff'] = wlLongStats['lev_va'].diff()

In [ ]:
wlLongStats

In [ ]:
wlLongStats['stdWL'] = wlLongStats[['lev_va','mean','std']].apply(lambda x: wa.avgMeths.stndrd(x),1 )

In [ ]:
wlLongStats['YRMO'] = wlLongStats[['Year','Month']].apply(lambda x: wa.avgMeths.yrmo(x),1)

In [ ]:
wlLongStats['date'] = wlLongStats[['Year','Month']].apply(lambda x: wa.avgMeths.adddate(x),1)

In [ ]:

wlLongStats.groupby(['Month'])['stdWL'].mean().to_frame().plot()
#wlLongStats.groupby(['Month'])['stdWL'].std().to_frame().plot()

#wlLongStats.groupby(['Month'])['stdWL'].(np.mean+np.std).plot()
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12])
plt.grid()

In [ ]:
import matplotlib.dates 
from matplotlib.dates import YearLocator

fig = plt.figure()
ax = fig.gca()

wlLongStatsGroups = wlLongStats.groupby(['date'])['stdWL'].agg({'mean':np.mean,'median':np.median,'standard':np.std, 
                                                                'cnt':(lambda x: np.count_nonzero(~np.isnan(x))), 'err':(lambda x: 1.96*np.std(x)/np.count_nonzero(~np.isnan(x)))})
wlLongStatsGroups2 = wlLongStats.groupby(['date'])['levDiff'].agg([np.mean,np.median,np.std])

wlLongStatsGroups['meanpluserr'] = wlLongStatsGroups['mean'] + wlLongStatsGroups['err']
wlLongStatsGroups['meanminuserr'] = wlLongStatsGroups['mean'] - wlLongStatsGroups['err']

wlLongSt = wlLongStatsGroups[wlLongStatsGroups['cnt']>2]

#x1 = pd.
x = wlLongSt.index
y = wlLongSt['mean']
ax.plot(x,y,label='Average Groundwater Level Variation')

ax.fill_between(wlLongSt.index, wlLongSt['meanpluserr'], wlLongSt['meanminuserr'], 
                 facecolor='blue', alpha=0.4, linewidth=0.5, label= "Std Error")
locator = YearLocator(2)
minlocator = YearLocator(1)
# Plotting stuff here ...
# Set major x ticks on Mondays.
plt.xlim('1/1/1945','1/1/2015')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_minor_locator(minlocator)
plt.grid(which='both')
plt.ylabel('z-score')
plt.xticks(rotation=45)
plt.legend()
plt.title('Average Groundwater Level Variation in Cache Valley, Utah and Idaho')

In [ ]:
wlLongStatsGroups

In [ ]:
wlLongStats

In [ ]:
plt.figure()
x = wlLongStatsGroups.index
y = wlLongStatsGroups['mean']
plt.plot(x,y)

In [ ]:
# designate variables
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10
from matplotlib.pyplot import cm 
x2 = USGS_Site_Info['amax'].astype(np.datetime64).values
x1 = USGS_Site_Info['amin'].astype(np.datetime64).values
z = USGS_Site_Info['size'].values.astype(np.int)
y = USGS_Site_Info.index.astype(np.int)
names = USGS_Site_Info.site_no.values

labs, tickloc, col = [], [], []

# create color iterator for multi-color lines in gantt chart
color=iter(cm.Dark2(np.linspace(0,1,len(y))))

plt.figure(figsize=[20,20])
fig, ax = plt.subplots()

# generate a line and line properties for each station
for i in range(len(y)):
    c=next(color)
    
    plt.hlines(i+1, x1[i], x2[i], label=y[i], color=c, linewidth=2)
    labs.append(str(names[i]).title()+" ( n= "+str(z[i])+")")
    tickloc.append(i+1)
    col.append(c)
plt.ylim(0,len(y)+1)
plt.yticks(tickloc, labs)

# create custom x labels
#plt.xticks(np.arange(datetime(np.min(x1).year,1,1),np.max(x2)+timedelta(days=365.25),timedelta(days=365.25*5)),rotation=45)
#plt.xlim(datetime(np.min(x1).year,1,1),np.max(x2)+timedelta(days=365.25))
plt.xlabel('Date')
plt.ylabel('USGS Official Station Id')
plt.grid()
plt.title('USGS Station Measurement Duration')
# color y labels to match lines
gytl = plt.gca().get_yticklabels()
for i in range(len(gytl)):
    gytl[i].set_color(col[i])
plt.tight_layout()

plt.savefig('E:\\PROJECTS\\UMAR\\Data\\USGS\\gantt.pdf')

In [ ]:
stationWL['Year']

In [ ]:
wlLongStats.to_csv('E:\\PROJECTS\\UMAR\\Data\\USGS\\AvgWLs_HUC_16010203.csv')

In [ ]:
z = stationWL.wlelev.values
x = stationWL.UTM_X.values
y = stationWL.UTM_Y.values

http://connor-johnson.com/2014/03/20/simple-kriging-in-python/

http://stackoverflow.com/questions/31124930/ckdtree-vs-dsearchn?lq=1

## Visualizing Data

## Exporting Figures

The following script will generate a series of box and whisker plots and save them in a pdf. It makes a box plot for each station, breaking the data into monthly intervals.  Make sure to change the directory name in the script so it ends up in a recognizable place on your computer.  

In [ ]:
# create dictionary of integers and their month equivalent
months = {'1':'Jan.', '2':'Feb.', '3':'Mar.', '4':'Apr.', '5':'May', '6':'Jun.', 
         '7':'Jul.', '8':'Aug.', '9':'Sep.', '10':'Oct.', '11':'Nov.', '12':'Dec.', 'Total':'Total'}
# create empty dictionary to hold pandas Dataframes
j = {}


with PdfPages(rootname + 'station_boxplots.pdf') as pdfs:
    ymax = 10000
    ymin = 0.01
    for i in range(len(sites)):
        # make a dataframe containing summary statistics and store it in the j dictionary
        j[sites[i]] = USGS_Site_Data.groupby('mon')[sites[i]].agg({'min':np.min, 'mean':np.mean, 
                                                                   'median':np.median, 'max':np.max, 'std':np.std, 
                                                                   'cnt':(lambda x: np.count_nonzero(~np.isnan(x)))}).reset_index()
        # make a list of the custom lables you will use for your boxplot; this one will show the number of samples used to make the plot
        labs = [months[(str(j[sites[i]]['mon'][b]))] + " (n=" + str(int(j[sites[i]]['cnt'][b])) + ")" for b in range(len(j[sites[i]]))]
        # designate the location of each custom label
        tickloc = [b+1 for b in range(len(j[sites[i]]['mon']))]
        
        plt.figure()
        USGS_Site_Data.boxplot(column=sites[i],by='mon', rot=70)
        strtdt = str(USGS_Site_Info.ix[sites[i],'start_date'])[0:10]
        findt = str(USGS_Site_Info.ix[sites[i],'fin_date'])[0:10]
        siteName = USGS_Site_Info.ix[sites[i],'name'].title() 
        plt.title( siteName + ' (' + sites[i] + ')  ' + strtdt + ' to ' + findt )
        plt.suptitle('')
        plt.yscale('log')
        plt.ylabel('Discharge (cfs)')
        plt.ylim((ymin,ymax))
        plt.xlabel('Month')
        # here is where your lists for the custom label come into play
        plt.xticks(tickloc, labs)
        
        pdfs.savefig()

        plt.close()
    # Save metadata of the pdf so you can find it later
    d = pdfs.infodict()
    d['Title'] = 'Monthly Station USGS Boxplots UMSS'
    d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
    d['Subject'] = 'Boxplots of several USGS Surface Stations'
    d['Keywords'] = 'USGS Surface NWIS Boxplot'
    d['CreationDate'] = datetime.today()
    d['ModDate'] = datetime.today()

Let's plot a few of the boxplots so you can see what they look like.

In [ ]:
for i in range(1,3):
    j[sites[i]] = USGS_Site_Data.groupby('mon')[sites[i]].agg([np.min, np.mean, np.median, np.max, np.std, np.size]).reset_index()
    # make a list of the custom lables you will use for your boxplot; this one will show the number of samples used to make the plot
    labs = [months[(str(j[sites[i]]['mon'][b]))] + " (n=" + str(int(j[sites[i]]['size'][b])) + ")" for b in range(len(j[sites[i]]))]
    # designate the location of each custom label
    tickloc = [b+1 for b in range(len(j[sites[i]]['mon']))]

    plt.figure()
    USGS_Site_Data.boxplot(column=sites[i],by='mon', rot=70)
    plt.title(USGS_Site_Info.ix[sites[i],'name'].title() + ' (' + sites[i] + ')')
    plt.suptitle('')
    plt.yscale('log')
    plt.ylabel('Discharge (cfs)')
    plt.ylim((ymin,ymax))
    plt.xlabel('Month')
    # here is where your lists for the custom label come into play
    plt.xticks(tickloc, labs)
    plt.show()
    plt.close()

This script will generate boxplots showing all of the station data.

In [ ]:
# This script summarizes discharge for all sites and limits the number of box plots on one graph to the n variable
j=0
with PdfPages(rootname + 'sum_boxplots.pdf') as pdf:
    while j < len(sites):
        ymax = 10000
        ymin = 0.01
        n=10
        # if statement allows for uneven number of sites on last page
        if j+n >= len(sites):
            plt.figure()
            USGS_Site_Data[sites[j:-1]].plot(kind='box')
            plt.title('Sites '+sites[j]+' to '+sites[-1] )
            plt.yscale('log')
            plt.xlabel('USGS Site')
            plt.xticks(rotation=45)
            plt.ylabel('discharge (cfs)')
            plt.ylim((ymin,ymax))
            pdf.savefig()
            plt.show()
            plt.close()
            j = j+n
        else:
            plt.figure()
            USGS_Site_Data[sites[j:j+n]].plot(kind='box')
            plt.title('Sites '+sites[j]+' to '+sites[j+n] )
            plt.yscale('log')
            plt.xlabel('USGS Site')
            plt.xticks(rotation=45)
            plt.ylabel('discharge (cfs)')
            plt.ylim((ymin,ymax))
            pdf.savefig()
            plt.show()
            plt.close()
            j = j+n
        # Save metadata of the pdf so you can find it later
        d = pdf.infodict()
        d['Title'] = 'Summary USGS Boxplots UMSS'
        d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
        d['Subject'] = 'Boxplots of several USGS Surface Stations'
        d['Keywords'] = 'USGS Surface NWIS Boxplot'
        d['CreationDate'] = datetime.today()
        d['ModDate'] = datetime.today()

We should also produce hydrographs of each station.

In [ ]:
xmax = USGS_Site_Data.index.astype(datetime).values[-1]
xmin = USGS_Site_Data.index.astype(datetime).values[0]

pdfs = PdfPages(rootname + 'station_hydrographs.pdf')
ymax = 10000
ymin = 0.1
for i in range(len(sites)):
    x = USGS_Site_Data.index.values
    y = USGS_Site_Data[sites[i]].values
    plt.figure()
    plt.plot(x,y)
    strtdt = str(USGS_Site_Info.ix[sites[i],'start_date'])[0:10]
    findt = str(USGS_Site_Info.ix[sites[i],'fin_date'])[0:10]
    siteName = USGS_Site_Info.ix[sites[i],'name'].title() 
    plt.title( siteName + ' (' + sites[i] + ')  ' + strtdt + ' to ' + findt )
    plt.suptitle('')
    plt.yscale('log')
    plt.ylabel('Discharge (cfs)')
    plt.ylim((ymin,ymax))
    plt.xlabel('Year')
    plt.xticks(np.arange(datetime(1905,1,1),xmax+timedelta(days=365.25),timedelta(days=365.25*5)),rotation=45)
    plt.xlim(xmin,xmax)
    pdfs.savefig()
    plt.close()
    # Save metadata of the pdf so you can find it later

d = pdfs.infodict()
d['Title'] = 'Monthly Station USGS Hydrographs UMSS'
d['Author'] = u'Paul C. Inkenbrandt\xe4nen'
d['Subject'] = 'Hydrograph of several USGS Surface Stations'
d['Keywords'] = 'USGS Surface NWIS Hydrograph'
d['CreationDate'] = datetime.today()
d['ModDate'] = datetime.today()

In [ ]:
pd.date_range(start=xmin, end=xmax, freq='5AS').year

In [ ]:
xmax = USGS_Site_Data.index[-1]
xmin = USGS_Site_Data.index[0]

plt.figure()
ticks = pd.date_range(start=xmin, end=xmax, freq='4AS')
USGS_Site_Data[sites[0:3]].plot(subplots=True,sharex=True,figsize=(10,8),logy=True, rot=90)
plt.xlim(xmin,xmax)
labs = pd.date_range(start=xmin, end=xmax, freq='4AS').year
plt.xticks(ticks,labs)
plt.show()
plt.close()

In [ ]:
def lumped_hydro(i1,i2):
    pdfs = PdfPages(rootname + 'station_hydrographs_lumped.pdf')
    plt.figure()
    ticks = pd.date_range(start=xmin, end=xmax, freq='4AS')
    USGS_Site_Data[sites[i1:i2]].plot(subplots=True, sharex=True, figsize=(10,24),logy=True, rot=90)
    plt.xlim(xmin,xmax)
    labs = pd.date_range(start=xmin, end=xmax, freq='4AS').year
    plt.xticks(ticks,labs)
    pdfs.savefig()
    plt.close()

In [ ]:
lumped_hydro(0,10)
lumped_hydro(10,20)

In [ ]:
lumped_hydro(20,30)
lumped_hydro(30,-1)

In [ ]:
def dic2df(dic,head):
    df = pd.DataFrame(data=dic)
    df = df.transpose()
    df.columns = [str(head)+'_var1',str(head)+'_var2',str(head)+'_var3',str(head)+'_var4',str(head)+'_r2',str(head)+'_err']
    return df

http://stackoverflow.com/questions/15408371/cumulative-distribution-plots-python <br>
http://hydroclimpy.sourceforge.net/installation.html

Run the following script if you want to see a map of your stations.  This assumes that you have the <a href=http://sourceforge.net/projects/matplotlib/files/matplotlib-toolkits/>Basemap package</a> installed.